- This notebook is for importing libraries and authenticating google search console by using service account.

In [0]:
!pip install requests_oauth2
!pip install searchconsole
!pip install google-api-python-client
!pip install google-auth-httplib2
!pip install google-auth-oauthlib
!pip install requests

  Using cached requests_oauth2-0.3.0-py3-none-any.whl
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-5c5a8be7-f520-4508-82df-52977e082a98/bin/python -m pip install --upgrade pip' command.
  Using cached searchconsole-0.0.4-py3-none-any.whl
  Using cached google_auth-2.15.0-py2.py3-none-any.whl (177 kB)
  Using cached google_api_python_client-2.70.0-py2.py3-none-any.whl (10.7 MB)
  Using cached google_auth_oauthlib-0.8.0-py2.py3-none-any.whl (19 kB)
  Using cached httplib2-0.21.0-py3-none-any.whl (96 kB)
  Using cached google_api_core-2.11.0-py3-none-any.whl (120 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl (10 kB)
  Using cached google_auth_httplib2-0.1.0-py2.py3-none-any.whl (9.3 kB)
  Using cached googleapis_common_protos-1.57.0-py2.py3-none-any.whl (217 kB)
  Using cached protobuf-4.21.12-cp37-abi3-manylinux2014_x86_64.whl (409 kB)
  Using cached cachetools-5.2.0-py3-none-any.whl (9.3 kB)
  Using cached rsa-4.9-py3-none-any.whl (34 kB)

In [0]:
import os
import searchconsole
import pandas as pd
from pyspark.sql import functions as f

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2 import service_account
import requests
from pyspark.sql import functions as f
import urllib.parse
from datetime import date, timedelta
import json

#### Authentication - service account

In [0]:
def gsc_connect(scopes, service_account_path):
    '''
    This function authenticate the usage of scope by getting authorization code based on interaction of browser,
    then it stores returned token file to DBFS
    Args:
        scopes: str
        cred_file: str - path of cred_file in DBFS
        token_file_path: str - path that you want to save token file
    Returns:
        service: object
    '''
    
    # if tokens exist, fetch them directly
    if os.path.exists(service_account_path):
        creds = service_account.Credentials.from_service_account_file(service_account_path, scopes=scopes)
    
    # authentication flow - needs manual user authorization to return authorization code
    service = build('webmasters', 'v3', credentials=creds)
    
    return service

In [0]:
# Google search console readonly scope
scopes = ['https://www.googleapis.com/auth/webmasters.readonly']

# the path of you uploading credentials file downloaded from Google developer console

credentials_path = "replace with you own DBFS path"

# credential file saved in dbfs
service_account_path = f"{credentials_path}"

#### Create startdate end date

In [0]:
def dates_calculation():
    '''
    This function prepare parameters for request - start_date and end_date
    Assuming performance needs to be pulled on monthly basis
    Args: 
        none
    Return: 
        start_date, end_date: str
    '''
    # enddate == yesterday
    enddate = (date.today() - timedelta(days=1)).strftime('%Y-%m-%d')
    
    # startdate == first day of previous month
    startdate = (date.today() - timedelta(days=1)).replace(day=1).strftime('%Y-%m-%d')
    return startdate, enddate

#### Post request

In [0]:
# request campaign level data from search console
def post_request(siteUrl, scopes, service_account_path, payload):
    '''
    This function post request and fetch data from Google Search Console
    Args: 
        none
    Return: 
        start_date, end_date: str
    '''
    # build service
    scopes = scopes
    service_account_path = service_account_path
    service = gsc_connect(scopes, service_account_path)
    
    # payload query parameters
    payload = payload
    # print(payload)
    # encode url to be used in para
    # siteUrl = urllib.parse.quote(siteUrl, safe="")

    # prepare query parameters and return dictionary
    data_response = service.searchanalytics().query(siteUrl=siteUrl, body=payload).execute()
    
    # convert to json
    json_obj = json.dumps(data_response)

    return json_obj